# Point-to-Mesh Similarity Analysis

This notebook demonstrates how to:
1. Load two meshes
2. Specify a particular point on the source mesh
3. Color-map the target mesh based on similarity to that point

The analysis uses diffusion features + DINO features to compute semantic similarity between mesh vertices.

## 1. Import Dependencies

In [28]:
import torch
from diff3f import get_features_per_vertex
from time import time
from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors
from dataloaders.mesh_container import MeshContainer
from diffusion import init_pipe
from dino import init_dino
from functional_map import compute_surface_map
import importlib
import meshplot as mp
from point_to_mesh_similarity import run_point_similarity_analysis, point_similarity_colormap, visualize_point_similarity, run_multi_point_correspondence_analysis
import numpy as np

%load_ext autoreload
%autoreload 2

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using device: cuda


## 2. Initialize Models

This step loads the diffusion pipeline and DINO model for feature extraction.

In [29]:
print("Initializing diffusion pipeline...")
pipe = init_pipe(device)

print("Initializing DINO model...")
dino_model = init_dino(device)

print("Models initialized successfully!")

Initializing diffusion pipeline...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Initializing DINO model...


Using cache found in /home/zixiliu/.cache/torch/hub/facebookresearch_dinov2_main


Models initialized successfully!


## 3. Load Meshes

Load your source and target meshes. You can modify the paths below to use your own meshes.

In [30]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_bottle_decomp.obj"

source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)


print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 9041 vertices, 17850 faces


## 4. Preview Meshes

Let's visualize the meshes before analysis to get familiar with their structure.

In [31]:
if 0:
    # Preview the meshes side by side
    d = mp.subplot(source_mesh.vert[:,0:3], source_mesh.face, s=[2, 2, 0])
    mp.subplot(target_mesh.vert[:,0:3], target_mesh.face, s=[2, 2, 1], data=d)
    print("Left: Source mesh | Right: Target mesh")

## 5. Configure Analysis Parameters

Set up the parameters for the similarity analysis.

In [32]:
# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

print(f"\nAnalysis Configuration:")
print(f"- Text prompt: '{prompt}'")
print(f"- Number of views: {num_views}")


p1 = [ 0.00886742, -0.04124315, -0.01754964]
p2 = [-0.01223545,  0.0363251 ,  0.00719686]
p3 = [-0.01694481,  0.03424783, -0.01735426]
p4 = [-0.01596961,  0.03464369, -0.02075735]
source_points_3d = np.array([p1, p2, p3, p4])


Analysis Configuration:
- Text prompt: 'a beaker'
- Number of views: 10


### To clean up meshes

In [33]:
if 0:
    from utils import clean_obj_file_format_only
    source_fixed = clean_obj_file_format_only("meshes/oakink_beaker_decomp2.obj")
    target_fixed = clean_obj_file_format_only("meshes/oakink_mug_decomp2.obj")
    
    # Now load the new target mesh with your MeshContainer
    source_mesh_path = "meshes/oakink_beaker_decomp2_fixed.obj"
    target_mesh_path = "meshes/oakink_mug_decomp2_fixed.obj"

### To clean up GPU cache

In [34]:
if 0:
    import torch
    
    # Clear cache of unused memory
    torch.cuda.empty_cache()
    
    # Optionally, collect unused memory from Python garbage collector
    import gc
    gc.collect()
    torch.cuda.empty_cache()

# Explore Re-mesh Open3D

In [35]:
if 0:
    mesh = o3d.io.read_triangle_mesh(target_mesh_path)
    
    # Create a point cloud with a uniform density using Poisson disk sampling
    # The number_of_points parameter controls the point density.
    # A higher number will result in a finer, more detailed final mesh.
    pcd = mesh.sample_points_poisson_disk(number_of_points=10000)
    
    # Optional: Visualize the point cloud to inspect the distribution
    o3d.visualization.draw_geometries([pcd])
    
    # The alpha parameter controls the size of the spheres used to create the mesh.
    # A smaller alpha will produce a tighter mesh that follows the point cloud more closely,
    # while a larger alpha will create a coarser mesh.
    alpha = 0.005
    new_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
    
    # Compute vertex normals for correct lighting and shading
    new_mesh.compute_vertex_normals()
    
    # Visualize the new mesh
    o3d.visualization.draw_geometries([new_mesh], mesh_show_back_face=True)
    
    # Optional: Save the new mesh
    # o3d.io.write_triangle_mesh("uniform_remeshed.ply", new_mesh)

    # Convert mesh to LineSet (wireframe representation)
    wireframe = o3d.geometry.LineSet.create_from_triangle_mesh(new_mesh)
    wireframe.paint_uniform_color([0, 0, 0])  # black wireframe
    
    # Show wireframe
    o3d.visualization.draw_geometries([wireframe])
    


## 6. Run Similarity Analysis

This is the main computation step. It will:
1. Extract features for both meshes using multiple camera views
2. Compute similarity between the specified source point and all target vertices
3. Generate the visualization

⚠️ **Note**: This step can take several minutes depending on mesh complexity and number of views.

In [36]:
remesh = True

if 0:
    source_mesh_path = "meshes/oakink_beaker_decomp2.obj"
    target_mesh_path = "meshes/oakink_bottle_decomp2.obj"
    num_views = 10  # Number of views for rendering (reduced for faster computation)

    if remesh: 
        from utils import remesh_mesh_pair
        source_mesh_path, target_mesh_path = remesh_mesh_pair(
            source_mesh_path,
            target_mesh_path,
            num_points=10000,
            alpha=0.005
        )

    
    source_mesh = MeshContainer().load_from_file(source_mesh_path)
    target_mesh = MeshContainer().load_from_file(target_mesh_path)

    
    print("Source mesh vert:", source_mesh.vert.shape)
    print("Target mesh vert:", target_mesh.vert.shape)
    
    # Run the complete analysis pipeline
    similarity_colors, raw_similarities, source_point_idx, closest_distance = run_point_similarity_analysis(
        source_mesh=source_mesh,
        target_mesh=target_mesh,
        source_point_3d=p1,
        device=device,
        pipe=pipe,
        dino_model=dino_model,
        prompt=prompt,
        num_views=num_views
    )
    
    print("\n✅ Analysis complete!")

In [37]:
remesh = False

if 0:
    source_mesh_path = "meshes/oakink_beaker_decomp2.obj"
    target_mesh_path = "meshes/oakink_bottle_decomp2.obj"
    num_views = 10  # Number of views for rendering (reduced for faster computation)

    if remesh: 
        from utils import remesh_mesh_pair
        source_mesh_path, target_mesh_path = remesh_mesh_pair(
            source_mesh_path,
            target_mesh_path,
            num_points=10000,
            alpha=0.005
        )

    
    source_mesh = MeshContainer().load_from_file(source_mesh_path)
    target_mesh = MeshContainer().load_from_file(target_mesh_path)

    
    print("Source mesh vert:", source_mesh.vert.shape)
    print("Target mesh vert:", target_mesh.vert.shape)
    
    # Run the complete analysis pipeline
    similarity_colors, raw_similarities, source_point_idx, closest_distance = run_point_similarity_analysis(
        source_mesh=source_mesh,
        target_mesh=target_mesh,
        source_point_3d=p1,
        device=device,
        pipe=pipe,
        dino_model=dino_model,
        prompt=prompt,
        num_views=num_views
    )
    
    print("\n✅ Analysis complete!")

# Multi point analysis

In [38]:
remesh=True

In [43]:
## NOTE: no difference between oakink_beaker_decomp2.obj and oakink_beaker_decomp.obj
source_mesh_path = "meshes/oakink_beaker_decomp.obj"
target_mesh_path = "meshes/oakink_bottle_decomp.obj"

if remesh: 
    from utils import remesh_mesh_pair
    source_mesh_path, target_mesh_path = remesh_mesh_pair(
        source_mesh_path,
        target_mesh_path,
        num_points=10000,
        alpha=0.005
    )
source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
num_views = 20  # Number of views for rendering (reduced for faster computation)

source_prompt = "beaker"
target_prompt = "bottle"
# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    source_prompt=source_prompt,
    target_prompt=target_prompt,
    num_views=num_views
)

print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
✅ Re-mesh complete: meshes/oakink_beaker_decomp_remesh.obj, meshes/oakink_bottle_decomp_remesh.obj
Source mesh: 9272 vertices, 18540 faces
Target mesh: 9102 vertices, 18200 faces
Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [ 0.00886742 -0.04124315 -0.01754964] -> Vertex 4953 (distance: 0.0043)
  Point 1: [-0.01223545  0.0363251   0.00719686] -> Vertex 3431 (distance: 0.0046)
  Point 2: [-0.01694481  0.03424783 -0.01735426] -> Vertex 6583 (distance: 0.0048)
  Point 3: [-0.01596961  0.03464369 -0.02075735] -> Vertex 1957 (distance: 0.0046)
Computing features for source mesh...


100%|██████████████████████████████| 20/20 [00:41<00:00,  2.07s/it]


Number of missing features:  385 . Copied features from nearest vertices.
Time taken in mins:  0.7364301204681396
Computing features for target mesh...


100%|██████████████████████████████| 20/20 [00:41<00:00,  2.07s/it]

Number of missing features:  1477 . Copied features from nearest vertices.
Time taken in mins:  0.7339926560719808
  Point 0: Source vertex 4953 -> Target vertex 4678 (similarity: 0.6211)
  Point 1: Source vertex 3431 -> Target vertex 4185 (similarity: 0.8721)
  Point 2: Source vertex 6583 -> Target vertex 6003 (similarity: 0.8623)
  Point 3: Source vertex 1957 -> Target vertex 8036 (similarity: 0.8696)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 4953 ↔ Target vertex 4678 (similarity: 0.6211)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 3431 ↔ Target vertex 4185 (similarity: 0.8721)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 6583 ↔ Target vertex 6003 (similarity: 0.8623)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 1957 ↔ Target vertex 8036 (similarity: 0.8696)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 9272 vertices + 24 marker vertices
   Target mesh: 9102 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.8063
  - Min similarity: 0.6211
  - Max similarity: 0.8721
  - Total correspondences: 4

 target_points_3d = 
 [[ 0.0098654  -0.0277576  -0.0362463 ]
 [-0.00548135  0.0309205  -0.0145579 ]
 [-0.0130016   0.0285901  -0.0313782 ]
 [-0.00885868  0.0301881  -0.0336474 ]]

✅ Analysis complete!


# A few more examples

In [44]:
num_views = 20


source_mesh_path = "meshes/oakink_beaker_decomp2.obj"
target_mesh_path = "meshes/oakink_mug_decomp2.obj"
if remesh: 
    from utils import remesh_mesh_pair
    source_mesh_path, target_mesh_path = remesh_mesh_pair(
        source_mesh_path,
        target_mesh_path,
        num_points=10000,
        alpha=0.005
    )
source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

source_prompt = "beaker"
target_prompt = "mug"
# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    source_prompt=source_prompt,
    target_prompt=target_prompt,
    num_views=num_views
)


print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
✅ Re-mesh complete: meshes/oakink_beaker_decomp2_remesh.obj, meshes/oakink_mug_decomp2_remesh.obj
Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [ 0.00886742 -0.04124315 -0.01754964] -> Vertex 7273 (distance: 0.0044)
  Point 1: [-0.01223545  0.0363251   0.00719686] -> Vertex 5159 (distance: 0.0044)
  Point 2: [-0.01694481  0.03424783 -0.01735426] -> Vertex 5997 (distance: 0.0045)
  Point 3: [-0.01596961  0.03464369 -0.02075735] -> Vertex 674

100%|██████████████████████████████| 20/20 [00:41<00:00,  2.08s/it]


Number of missing features:  393 . Copied features from nearest vertices.
Time taken in mins:  0.7419969995816549
Computing features for target mesh...


100%|██████████████████████████████| 20/20 [00:38<00:00,  1.90s/it]

Number of missing features:  652 . Copied features from nearest vertices.
Time taken in mins:  0.6763251145680745
  Point 0: Source vertex 7273 -> Target vertex 2553 (similarity: 0.7065)
  Point 1: Source vertex 5159 -> Target vertex 249 (similarity: 0.7480)
  Point 2: Source vertex 5997 -> Target vertex 893 (similarity: 0.7520)
  Point 3: Source vertex 6743 -> Target vertex 893 (similarity: 0.7344)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7273 ↔ Target vertex 2553 (similarity: 0.7065)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 5159 ↔ Target vertex 249 (similarity: 0.7480)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 5997 ↔ Target vertex 893 (similarity: 0.7520)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 6743 ↔ Target vertex 893 (similarity: 0.7344)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 9267 vertices + 24 marker vertices
   Target mesh: 8072 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.7352
  - Min similarity: 0.7065
  - Max similarity: 0.7520
  - Total correspondences: 4

 target_points_3d = 
 [[ 0.00550012 -0.0579513   0.0121382 ]
 [-0.0198648   0.0201408   0.00062377]
 [-0.0269952   0.015379   -0.00771644]
 [-0.0269952   0.015379   -0.00771644]]

✅ Analysis complete!


In [46]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp2.obj"
target_mesh_path = "meshes/oakink_wineglass_decomp2.obj"
if remesh: 
    from utils import remesh_mesh_pair
    source_mesh_path, target_mesh_path = remesh_mesh_pair(
        source_mesh_path,
        target_mesh_path,
        num_points=10000,
        alpha=0.005
    )
    
source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

# Analysis parameters
num_views = 20  # Number of views for rendering (reduced for faster computation)
source_prompt = "beaker"
target_prompt = "wineglass"
# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    source_prompt=source_prompt,
    target_prompt=target_prompt,
    num_views=num_views
)

print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh

100%|██████████████████████████████| 20/20 [00:41<00:00,  2.07s/it]


Number of missing features:  385 . Copied features from nearest vertices.
Time taken in mins:  0.7374135851860046
Computing features for target mesh...


100%|██████████████████████████████| 20/20 [00:43<00:00,  2.18s/it]

Number of missing features:  217 . Copied features from nearest vertices.
Time taken in mins:  0.7692441344261169
  Point 0: Source vertex 5829 -> Target vertex 2200 (similarity: 0.6177)
  Point 1: Source vertex 3200 -> Target vertex 4005 (similarity: 0.6943)
  Point 2: Source vertex 3192 -> Target vertex 4567 (similarity: 0.7153)
  Point 3: Source vertex 5945 -> Target vertex 4005 (similarity: 0.7109)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 5829 ↔ Target vertex 2200 (similarity: 0.6177)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 3200 ↔ Target vertex 4005 (similarity: 0.6943)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 3192 ↔ Target vertex 4567 (similarity: 0.7153)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 5945 ↔ Target vertex 4005 (similarity: 0.7109)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 9262 vertices + 24 marker vertices
   Target mesh: 8490 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.6846
  - Min similarity: 0.6177
  - Max similarity: 0.7153
  - Total correspondences: 4

 target_points_3d = 
 [[ 0.0080457  -0.0276895   0.0691929 ]
 [ 0.00808573  0.0352702   0.0468748 ]
 [ 0.011697    0.0340958   0.044947  ]
 [ 0.00808573  0.0352702   0.0468748 ]]

✅ Analysis complete!


#### Note the bowl has issue with point distribution for re-meshing. But the original has normal triangles

In [61]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp2.obj"
target_mesh_path = "meshes/oakink_bowl_decomp2.obj"
if 0: 
    from utils import remesh_mesh_pair
    source_mesh_path, target_mesh_path = remesh_mesh_pair(
        source_mesh_path,
        target_mesh_path,
        num_points=10000,
        alpha=0.005
    )
source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
num_views = 10  # Number of views for rendering (reduced for faster computation)
source_prompt = "beaker"
target_prompt = "bowl"
# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    source_prompt=source_prompt,
    target_prompt=target_prompt,
    num_views=num_views
)

print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 15566 vertices, 30916 faces
Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [ 0.00886742 -0.04124315 -0.01754964] -> Vertex 5663 (distance: 0.0100)
  Point 1: [-0.01223545  0.0363251   0.00719686] -> Vertex 9145 (distance: 0.0116)
  Point 2: [-0.01694481  0.03424783 -0.01735426] -> Vertex 2712 (distance: 0.0060)
  Point 3: [-0.01596961  0.03464369 -0.02075735] -> Vertex 2740 (distance: 0.0053)
Computing features for source mesh...


100%|██████████████████████████████| 10/10 [00:25<00:00,  2.55s/it]


Number of missing features:  982 . Copied features from nearest vertices.
Time taken in mins:  0.4563605984052022
Computing features for target mesh...


100%|██████████████████████████████| 10/10 [00:22<00:00,  2.30s/it]

Number of missing features:  1371 . Copied features from nearest vertices.
Time taken in mins:  0.42268147071202594
  Point 0: Source vertex 5663 -> Target vertex 13787 (similarity: 0.4062)
  Point 1: Source vertex 9145 -> Target vertex 3989 (similarity: 0.5610)
  Point 2: Source vertex 2712 -> Target vertex 4799 (similarity: 0.3696)
  Point 3: Source vertex 2740 -> Target vertex 4108 (similarity: 0.3711)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 5663 ↔ Target vertex 13787 (similarity: 0.4062)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9145 ↔ Target vertex 3989 (similarity: 0.5610)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 2712 ↔ Target vertex 4799 (similarity: 0.3696)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 2740 ↔ Target vertex 4108 (similarity: 0.3711)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 15566 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.4270
  - Min similarity: 0.3696
  - Max similarity: 0.5610
  - Total correspondences: 4

 target_points_3d = 
 [[-0.00372535 -0.06896716  0.00819176]
 [-0.02775519  0.05429005  0.01184506]
 [-0.00955814  0.0672548   0.00519297]
 [-0.01615746  0.07144749  0.02109833]]

✅ Analysis complete!


In [68]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp2.obj"
target_mesh_path = "meshes/oakink_mug2_decomp.obj"
if remesh: 
    from utils import remesh_mesh_pair
    source_mesh_path, target_mesh_path = remesh_mesh_pair(
        source_mesh_path,
        target_mesh_path,
        num_points=10000,
        alpha=0.005
    )
source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

# Analysis parameters
num_views = 10  # Number of views for rendering (reduced for faster computation)
source_prompt = "beaker"
target_prompt = "mug"
# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    source_prompt=source_prompt,
    target_prompt=target_prompt,
    num_views=num_views
)

print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh

100%|██████████████████████████████| 10/10 [00:22<00:00,  2.23s/it]


Number of missing features:  542 . Copied features from nearest vertices.
Time taken in mins:  0.3980675935745239
Computing features for target mesh...


100%|██████████████████████████████| 10/10 [00:19<00:00,  1.93s/it]

Number of missing features:  1042 . Copied features from nearest vertices.
Time taken in mins:  0.3413896958033244
  Point 0: Source vertex 2281 -> Target vertex 5183 (similarity: 0.6455)
  Point 1: Source vertex 6748 -> Target vertex 2440 (similarity: 0.6323)
  Point 2: Source vertex 5185 -> Target vertex 3391 (similarity: 0.6118)
  Point 3: Source vertex 3559 -> Target vertex 2954 (similarity: 0.6284)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 2281 ↔ Target vertex 5183 (similarity: 0.6455)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 6748 ↔ Target vertex 2440 (similarity: 0.6323)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 5185 ↔ Target vertex 3391 (similarity: 0.6118)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 3559 ↔ Target vertex 2954 (similarity: 0.6284)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 9283 vertices + 24 marker vertices
   Target mesh: 7340 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.6295
  - Min similarity: 0.6118
  - Max similarity: 0.6455
  - Total correspondences: 4

 target_points_3d = 
 [[ 0.0117988  -0.0609269  -0.0114519 ]
 [-0.0249436   0.0196088   0.00441126]
 [-0.0253744   0.0209693  -0.0116892 ]
 [-0.0312149   0.0149998  -0.00086065]]

✅ Analysis complete!


#### Note the vase has issue with point distribution for re-meshing. 

In [60]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp2.obj"
target_mesh_path = "meshes/oakink_vase_decomp.obj"
if 0: 
    from utils import remesh_mesh_pair
    source_mesh_path, target_mesh_path = remesh_mesh_pair(
        source_mesh_path,
        target_mesh_path,
        num_points=10000,
        alpha=0.005
    )
source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
num_views = 10  # Number of views for rendering (reduced for faster computation)
source_prompt = "beaker"
target_prompt = "vase"
# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    source_prompt=source_prompt,
    target_prompt=target_prompt,
    num_views=num_views
)

print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 7845 vertices, 15610 faces
Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [ 0.00886742 -0.04124315 -0.01754964] -> Vertex 5663 (distance: 0.0100)
  Point 1: [-0.01223545  0.0363251   0.00719686] -> Vertex 9145 (distance: 0.0116)
  Point 2: [-0.01694481  0.03424783 -0.01735426] -> Vertex 2712 (distance: 0.0060)
  Point 3: [-0.01596961  0.03464369 -0.02075735] -> Vertex 2740 (distance: 0.0053)
Computing features for source mesh...


100%|██████████████████████████████| 10/10 [00:25<00:00,  2.55s/it]


Number of missing features:  982 . Copied features from nearest vertices.
Time taken in mins:  0.45577502250671387
Computing features for target mesh...


100%|██████████████████████████████| 10/10 [00:22<00:00,  2.25s/it]

Number of missing features:  572 . Copied features from nearest vertices.
Time taken in mins:  0.3956053137779236
  Point 0: Source vertex 5663 -> Target vertex 892 (similarity: 0.5967)
  Point 1: Source vertex 9145 -> Target vertex 4976 (similarity: 0.4028)
  Point 2: Source vertex 2712 -> Target vertex 4647 (similarity: 0.6406)
  Point 3: Source vertex 2740 -> Target vertex 4647 (similarity: 0.6357)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 5663 ↔ Target vertex 892 (similarity: 0.5967)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9145 ↔ Target vertex 4976 (similarity: 0.4028)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 2712 ↔ Target vertex 4647 (similarity: 0.6406)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 2740 ↔ Target vertex 4647 (similarity: 0.6357)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 7845 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.5690
  - Min similarity: 0.4028
  - Max similarity: 0.6406
  - Total correspondences: 4

 target_points_3d = 
 [[ 0.01797572 -0.0314595  -0.03472932]
 [-0.00011542  0.01414152  0.06510469]
 [ 0.00510972  0.02760778 -0.00388352]
 [ 0.00510972  0.02760778 -0.00388352]]

✅ Analysis complete!


In [56]:
if 0:
    import open3d as o3d
    mesh = o3d.io.read_triangle_mesh(target_mesh_path)
    
    # Create a point cloud with a uniform density using Poisson disk sampling
    # The number_of_points parameter controls the point density.
    # A higher number will result in a finer, more detailed final mesh.
    pcd = mesh.sample_points_poisson_disk(number_of_points=1000)
    
    # Optional: Visualize the point cloud to inspect the distribution
    o3d.visualization.draw_geometries([pcd])
    
    # The alpha parameter controls the size of the spheres used to create the mesh.
    # A smaller alpha will produce a tighter mesh that follows the point cloud more closely,
    # while a larger alpha will create a coarser mesh.
    alpha = 0.005
    new_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
    
    # Compute vertex normals for correct lighting and shading
    new_mesh.compute_vertex_normals()
    
    # Visualize the new mesh
    o3d.visualization.draw_geometries([new_mesh], mesh_show_back_face=True)
    
    # Optional: Save the new mesh
    # o3d.io.write_triangle_mesh("uniform_remeshed.ply", new_mesh)

    # Convert mesh to LineSet (wireframe representation)
    wireframe = o3d.geometry.LineSet.create_from_triangle_mesh(new_mesh)
    wireframe.paint_uniform_color([0, 0, 0])  # black wireframe
    
    # Show wireframe
    o3d.visualization.draw_geometries([wireframe])
    


In [70]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp2.obj"
target_mesh_path = "meshes/oakink_jar_decomp.obj"
if 0: 
    from utils import remesh_mesh_pair
    source_mesh_path, target_mesh_path = remesh_mesh_pair(
        source_mesh_path,
        target_mesh_path,
        num_points=10000,
        alpha=0.005
    )
    
source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)

print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

# Analysis parameters
num_views = 20  # Number of views for rendering (reduced for faster computation)
source_prompt = "beaker"
target_prompt = "jar"
# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance, target_points_3d = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    source_prompt=source_prompt,
    target_prompt=target_prompt,
    num_views=num_views
)

print("\n target_points_3d = \n", target_points_3d)
print("\n✅ Analysis complete!")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 13513 vertices, 26746 faces
Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [ 0.00886742 -0.04124315 -0.01754964] -> Vertex 5663 (distance: 0.0100)
  Point 1: [-0.01223545  0.0363251   0.00719686] -> Vertex 9145 (distance: 0.0116)
  Point 2: [-0.01694481  0.03424783 -0.01735426] -> Vertex 2712 (distance: 0.0060)
  Point 3: [-0.01596961  0.03464369 -0.02075735] -> Vertex 2740 (distance: 0.0053)
Computing features for source mesh...


100%|██████████████████████████████| 20/20 [00:50<00:00,  2.50s/it]


Number of missing features:  521 . Copied features from nearest vertices.
Time taken in mins:  0.8935544927914937
Computing features for target mesh...


100%|██████████████████████████████| 20/20 [00:51<00:00,  2.60s/it]

Number of missing features:  1067 . Copied features from nearest vertices.
Time taken in mins:  0.9328925530115764
  Point 0: Source vertex 5663 -> Target vertex 4003 (similarity: 0.7969)
  Point 1: Source vertex 9145 -> Target vertex 8806 (similarity: 0.3945)
  Point 2: Source vertex 2712 -> Target vertex 71 (similarity: 0.8716)
  Point 3: Source vertex 2740 -> Target vertex 71 (similarity: 0.8643)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 5663 ↔ Target vertex 4003 (similarity: 0.7969)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9145 ↔ Target vertex 8806 (similarity: 0.3945)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 2712 ↔ Target vertex 71 (similarity: 0.8716)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 2740 ↔ Target vertex 71 (similarity: 0.8643)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 13513 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.7318
  - Min similarity: 0.3945
  - Max similarity: 0.8716
  - Total correspondences: 4

 target_points_3d = 
 [[ 0.03289173 -0.05716169 -0.0028039 ]
 [ 0.00190182 -0.04391325  0.07511032]
 [ 0.00946672  0.06469485  0.0045614 ]
 [ 0.00946672  0.06469485  0.0045614 ]]

✅ Analysis complete!


## 7. Analyze Results

Let's examine the similarity scores and find the most similar regions.

In [ ]:
# Find the most similar points on target mesh
top_k = 10
most_similar_indices = np.argsort(raw_similarities)[-top_k:][::-1]
least_similar_indices = np.argsort(raw_similarities)[:top_k]

print(f"📊 Similarity Statistics:")
print(f"- Min similarity: {raw_similarities.min():.4f}")
print(f"- Max similarity: {raw_similarities.max():.4f}")
print(f"- Mean similarity: {raw_similarities.mean():.4f}")
print(f"- Std similarity: {raw_similarities.std():.4f}")

print(f"\n🔥 Top {top_k} most similar vertices on target mesh:")
for i, idx in enumerate(most_similar_indices):
    print(f"  {i+1:2d}. Vertex {idx:5d}: similarity = {raw_similarities[idx]:.4f}")

print(f"\n❄️  Bottom {top_k} least similar vertices on target mesh:")
for i, idx in enumerate(least_similar_indices):
    print(f"  {i+1:2d}. Vertex {idx:5d}: similarity = {raw_similarities[idx]:.4f}")

## 8. Visualize Similarity Distribution

Plot the distribution of similarity scores to understand the data better.

In [ ]:
import matplotlib.pyplot as plt 

# Plot similarity distribution
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(raw_similarities, bins=50, alpha=0.7, edgecolor='black')
plt.axvline(raw_similarities.mean(), color='red', linestyle='--', label=f'Mean: {raw_similarities.mean():.3f}')
plt.axvline(raw_similarities.max(), color='green', linestyle='--', label=f'Max: {raw_similarities.max():.3f}')
plt.xlabel('Similarity Score')
plt.ylabel('Frequency')
plt.title('Distribution of Similarity Scores')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(sorted(raw_similarities, reverse=True), linewidth=2)
plt.xlabel('Vertex Rank (sorted by similarity)')
plt.ylabel('Similarity Score')
plt.title('Similarity Scores (Ranked)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Interactive Visualization

The main result visualization showing:
- **Left**: Source mesh (gray) with the selected point highlighted in red
- **Right**: Target mesh colored by similarity (warmer colors = more similar)

You can interact with the 3D visualization to rotate and zoom the meshes.

In [ ]:
print("🎨 Interactive Visualization:")
print("- Source mesh (left): Gray with red point showing the reference vertex")
print("- Target mesh (right): Colored by similarity (warmer colors = more similar)")
print("- Use mouse to rotate and zoom the 3D view")

# The visualization was already created by run_point_similarity_analysis
# But let's create it again with custom colormap options
visualize_point_similarity(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_point_idx=source_point_idx,
    similarity_colors=similarity_colors,
)

## 10. Experiment with Different Points

Try analyzing different points on the source mesh to see how similarity patterns change.

In [ ]:
# Experiment with different source points
# You can change these indices and re-run this cell
experiment_points = [100, 500, 1500, 2000]  # Different vertex indices to try

for point_idx in experiment_points:
    if point_idx < len(source_mesh.vert):
        print(f"\n🔬 Experimenting with source vertex {point_idx}...")

        # Quick analysis with fewer views for faster computation
        exp_colors, exp_similarities = point_similarity_colormap(
            device=device,
            pipe=pipe,
            dino_model=dino_model,
            source_mesh=source_mesh,
            target_mesh=target_mesh,
            source_point_idx=point_idx,
            prompt=prompt,
            num_views=10  # Fewer views for quick experiment
        )

        # Show quick stats
        top_match = np.argmax(exp_similarities)
        print(f"  Best match: vertex {top_match} (similarity: {exp_similarities[top_match]:.4f})")

        # Visualize
        visualize_point_similarity(
            source_mesh=source_mesh,
            target_mesh=target_mesh,
            source_point_idx=point_idx,
            similarity_colors=exp_colors,
            colormap='viridis'
        )
    else:
        print(f"Skipping vertex {point_idx} (mesh only has {len(source_mesh.vert)} vertices)")

## 11. Save Results (Optional)

Save the similarity data for later analysis.

In [ ]:
# Save results to files
output_prefix = f"similarity_results_vertex_{source_point_idx}"

# Save similarity scores
np.save(f"{output_prefix}_similarities.npy", raw_similarities)
np.save(f"{output_prefix}_colors.npy", similarity_colors)

# Save analysis summary
summary = {
    'source_mesh_path': source_mesh_path,
    'target_mesh_path': target_mesh_path,
    'source_point_idx': source_point_idx,
    'prompt': prompt,
    'num_views': num_views,
    'min_similarity': float(raw_similarities.min()),
    'max_similarity': float(raw_similarities.max()),
    'mean_similarity': float(raw_similarities.mean()),
    'std_similarity': float(raw_similarities.std()),
    'top_matches': [int(idx) for idx in most_similar_indices[:5]]
}

import json
with open(f"{output_prefix}_summary.json", 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✅ Results saved:")
print(f"  - Similarities: {output_prefix}_similarities.npy")
print(f"  - Colors: {output_prefix}_colors.npy")
print(f"  - Summary: {output_prefix}_summary.json")

## Summary

This notebook demonstrated how to:

1. ✅ Load and visualize 3D meshes
2. ✅ Extract semantic features using diffusion + DINO models
3. ✅ Compute similarity between a specific source point and all target vertices
4. ✅ Visualize results with interactive 3D plots
5. ✅ Analyze similarity distributions and find best matches
6. ✅ Experiment with different source points
7. ✅ Save results for future analysis

### Next Steps

- Try different mesh pairs
- Experiment with different prompts
- Adjust the number of views for quality vs. speed trade-offs
- Use the saved similarity data for further analysis or applications